In [2]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

import numpy as np
import wandb

In [3]:
class CNN(nn.Module):
  def __init__(self, in_features = 1, out_features = [16, 32], dropout = 0.1):
    super(CNN, self).__init__()

    self.model = nn.Sequential(
        nn.Conv2d(in_channels=in_features, out_channels=out_features[0], kernel_size=5, stride = 1, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(in_channels=out_features[0], out_channels=out_features[1], kernel_size=5, stride = 1, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout(p = dropout)
    )

    self.fc = nn.Linear(out_features[1] * 4*4, 10)

  def forward(self, x):
    output = self.model(x)
    output = output.view(output.size(0), -1)
    return self.fc(output)

In [5]:
hyperparameter_defaults = dict(
    dropout = 0.5,
    channels_one = 16,
    channels_two = 32,
    batch_size = 100,
    learning_rate = 0.001,
    epochs = 10,
    )

####################wandb Initialize#################################
wandb.init(config = hyperparameter_defaults, project = "cnn-f_mnist")
config = wandb.config
#####################################################################

train_transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.RandomHorizontalFlip(0.5),
      transforms.Normalize((0.5,), (0.5,)),
  ])

test_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,)),
])

train_datasets = torchvision.datasets.FashionMNIST(root ='./', train=True, download=False, transform=train_transform)
test_datasets = torchvision.datasets.FashionMNIST(root ='./', train=False, download=False, transform=test_transform)

train_loader = torch.utils.data.DataLoader(dataset=train_datasets,
                                               batch_size=config.batch_size,
                                               shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_datasets,
                                              batch_size=config.batch_size,
                                              shuffle=False)

classes = [ "T-shirt or top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Boot"]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNN(in_features=1, out_features=[config.channels_one, config.channels_two], dropout=config.dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

def train(model, train_loader,criterion, optimizer, epoch, device):
  model.train()
  running_loss = 0.0
  dataset_size = 0

  pbar = tqdm(enumerate(train_loader), total=len(train_loader))
  for idx, (imgs, labels) in pbar:
    imgs = imgs.to(device).float()
    labels = labels.to(device).long()
    batch_size = imgs.size(0)

    optimizer.zero_grad()
    pred = model(imgs)
    loss = criterion(pred, labels)
    loss.backward()
    optimizer.step()

    running_loss += (loss.item() * batch_size)
    dataset_size += batch_size
    epoch_loss = running_loss / dataset_size
    pbar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss)

  return epoch_loss

def eval(model, test_loader, criterion, optimizer, epoch, device):
  model.eval()
  running_loss = 0.0
  dataset_size = 0
  correct = 0.0
  
  image_preds_all = []
  image_targets_all = []

  pbar = tqdm(enumerate(test_loader), total=len(test_loader))
  for idx, (imgs, labels) in pbar:
    imgs = imgs.to(device).float()
    labels = labels.to(device).long()
    batch_size = imgs.size(0)

    pred = model(imgs)
    image_preds_all += [torch.argmax(pred, 1).detach().cpu().numpy()]
    image_targets_all += [labels.detach().cpu().numpy()]
    loss = criterion(pred, labels)
    
    running_loss += (loss.item() * batch_size)
    dataset_size += batch_size
    epoch_loss = running_loss / dataset_size
    pbar.set_postfix(Epoch=epoch, valid_Loss=epoch_loss)

  image_preds_all = np.concatenate(image_preds_all)
  image_targets_all = np.concatenate(image_targets_all)
  valid_Acc = (image_preds_all==image_targets_all).mean()
  
  return epoch_loss, valid_Acc

def main(model, train_loader,test_loader ,optimizer, device, epochs):
  wandb.watch(model, log = "all", log_freq = 100)

  for epoch in range(1, epochs+1):
    train_epoch_loss = train(model, train_loader, criterion, optimizer, device = device, epoch = epoch)
    with torch.no_grad():
      val_epoch_loss, val_acc = eval(model, test_loader, criterion, optimizer, epoch=epoch, device=device)

    wandb.log({"Train Loss": train_epoch_loss})
    wandb.log({"Valid Loss": val_epoch_loss})
    wandb.log({"Valid Acc": val_acc})

main(model, train_loader=train_loader, test_loader =test_loader, optimizer=optimizer, device = device, epochs=config.epochs)


  0%|          | 0/600 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 100/100 [00:03<00:00, 33.11it/s, Epoch=10, valid_Loss=0.313]
